In [23]:
from flask import Flask,render_template, request, redirect
from __future__ import print_function
import os
import cv2
import imageio
import PIL.Image
import numpy as np
import tensorflow as tf
from button_detection import ButtonDetector
from character_recognition import CharacterRecognizer

In [2]:
def warm_up(detector, recognizer):
    image = imageio.imread('./test_panels/1.jpg')
    button = imageio.imread('./test_buttons/0_0.png')
    detector.predict(image)
    recognizer.predict(button)

In [3]:
def predict(img_np, recognizer, detector):
    boxes, scores, _ = detector.predict(img_np, True)
    button_patches, button_positions, _ = button_candidates(boxes, scores, img_np)

    for button_img, button_pos in zip(button_patches, button_positions):
        button_text, button_score, button_draw =recognizer.predict(button_img, draw=True)
        x_min, y_min, x_max, y_max = button_pos
        button_rec = cv2.resize(button_draw, (x_max-x_min, y_max-y_min))
        detector.image_show[y_min+6:y_max-6, x_min+6:x_max-6] = button_rec[6:-6, 6:-6]
    return detector.image_show

In [4]:
def button_candidates(boxes, scores, image):
    img_height = image.shape[0]
    img_width = image.shape[1]

    button_scores = []
    button_patches = []
    button_positions = []

    for box, score in zip(boxes, scores):
        if score < 0.5: continue

        y_min = int(box[0] * img_height)
        x_min = int(box[1] * img_width)
        y_max = int(box[2] * img_height)
        x_max = int(box[3] * img_width)

        button_patch = image[y_min: y_max, x_min: x_max]
        button_patch = cv2.resize(button_patch, (180, 180))

        button_scores.append(score)
        button_patches.append(button_patch)
        button_positions.append([x_min, y_min, x_max, y_max])
    return button_patches, button_positions, button_scores

In [5]:
detector = ButtonDetector()
recognizer = CharacterRecognizer(verbose=False)
warm_up(detector, recognizer)    

button detector initialized!
character recognizer initialized!


In [25]:
app = Flask(__name__)  

@app.route("/")
def index():
    return render_template("./upload.html")

@app.route("/run", methods=['GET', 'POST'])
def run():
    if request.method == 'POST':
        f = request.files['file']
        img = PIL.Image.open(f)
        img_np = np.array(img)
        if img_np.shape[-1] == 4:
            img_np = img_np[:,:,:3]
        tmp = predict(img_np, recognizer, detector)
        print(type(tmp))
        result_img = PIL.Image.fromarray(tmp)
        result_img.save("static/temp/temp.png")
        return redirect("/run")
    return render_template('run.html', image_file="temp/temp.png")

@app.after_request
def add_header(r):
    """
    Add headers to both force latest IE rendering engine or Chrome Frame,
    and also to cache the rendered page for 10 minutes.
    """
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

In [18]:
host_addr = "127.0.0.1"
port_num = "8080"

In [26]:
app.run(host=host_addr, port=port_num)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Aug/2019 17:36:53] "GET / HTTP/1.1" 200 -


<type 'numpy.ndarray'>


127.0.0.1 - - [07/Aug/2019 17:37:04] "POST /run HTTP/1.1" 302 -
127.0.0.1 - - [07/Aug/2019 17:37:04] "GET /run HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2019 17:37:04] "GET /static/temp/temp.png HTTP/1.1" 200 -


<type 'numpy.ndarray'>


127.0.0.1 - - [07/Aug/2019 17:47:03] "POST /run HTTP/1.1" 302 -
127.0.0.1 - - [07/Aug/2019 17:47:03] "GET /run HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2019 17:47:03] "GET /static/temp/temp.png HTTP/1.1" 200 -


<type 'numpy.ndarray'>


127.0.0.1 - - [07/Aug/2019 17:58:14] "POST /run HTTP/1.1" 302 -
127.0.0.1 - - [07/Aug/2019 17:58:14] "GET /run HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2019 17:58:14] "GET /static/temp/temp.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2019 17:58:21] "GET /static/temp/temp.png HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2019 19:04:00] "GET /static/temp/temp.png HTTP/1.1" 200 -
